In [ ]:
#Cell 1 – Imports + Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load data from baseline module
train = pd.read_csv("../baseline_prophet_forecast/data/train.csv")
features = pd.read_csv("../baseline_prophet_forecast/data/features.csv")
stores = pd.read_csv("../baseline_prophet_forecast/data/stores.csv")

df = (
    train
    .merge(features, on=["Store", "Date", "IsHoliday"])
    .merge(stores, on="Store")
)

df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(["Store", "Dept", "Date"])
df.head()


In [ ]:
#Cell 2 – Define SKU + Basic Aggregates
df["sku"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

# Basic SKU-level statistics on Weekly_Sales
sku_stats = df.groupby("sku")["Weekly_Sales"].agg(
    mean="mean",
    std="std",
    median="median",
    max="max",
    min="min"
).reset_index()

# Coefficient of variation (volatility normalized by level)
sku_stats["cv"] = sku_stats["std"] / sku_stats["mean"]
sku_stats.replace([np.inf, -np.inf], np.nan, inplace=True)
sku_stats.fillna(0, inplace=True)

sku_stats.head()


In [ ]:
#Cell 3 – Weekly Aggregation for Trend & Seasonality
# Add week of year
df["weekofyear"] = df["Date"].dt.isocalendar().week.astype(int)

# Average weekly sales per sku per week
weekly = (
    df.groupby(["sku", "weekofyear"])["Weekly_Sales"]
    .mean()
    .reset_index()
)

weekly.head()


In [ ]:
#Pivot to get a matrix of shape (sku × week):
pivot = weekly.pivot(index="sku", columns="weekofyear", values="Weekly_Sales").fillna(0)
pivot.head()


In [ ]:
#Trend proxy: compare last 4 weeks vs first 4 weeks:
# Ensure consistent week ordering
pivot = pivot.sort_index(axis=1)

early_weeks = pivot.iloc[:, :4].mean(axis=1)
late_weeks = pivot.iloc[:, -4:].mean(axis=1)

trend = late_weeks - early_weeks            # >0 = increasing, <0 = declining
seasonality_strength = pivot.var(axis=1)    # variance across weeks as a simple proxy


In [ ]:
#Create a DataFrame from these:
trend_df = pd.DataFrame({
    "sku": pivot.index,
    "trend": trend.values,
    "seasonality_strength": seasonality_strength.values
})

trend_df.head()

In [ ]:
#Cell 4 – Merge Features into One Table
merged = sku_stats.merge(trend_df, on="sku", how="left")
merged.fillna(0, inplace=True)

merged.head()


In [ ]:
#Feature columns we’ll use for clustering:
feature_cols = ["mean", "std", "cv", "trend", "seasonality_strength"]
X = merged[feature_cols]


In [ ]:
#Cell 5 – Scale Features & Run K-Means
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 4  # you can experiment with 3–6
kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
merged["cluster"] = kmeans.fit_predict(X_scaled)

merged["cluster"].value_counts().sort_index()


In [ ]:
#Cell 6 – Visualize Clusters: Trend vs Volatility
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=merged,
    x="trend",
    y="cv",
    hue="cluster",
    palette="Set2",
    s=50
)
plt.axvline(0, color="grey", linestyle="--", linewidth=1)
plt.title("SKU Segmentation: Trend vs Volatility (CV)")
plt.xlabel("Trend (Late Weeks - Early Weeks)")
plt.ylabel("Coefficient of Variation (cv)")
plt.tight_layout()
plt.savefig("images/sku_clusters_trend_cv.png")
plt.show()


In [ ]:
#Cell 7 – Cluster Profiles Heatmap
cluster_profile = merged.groupby("cluster")[feature_cols].mean()

plt.figure(figsize=(8, 6))
sns.heatmap(cluster_profile, annot=True, fmt=".1f", cmap="Blues")
plt.title("Cluster Profiles (Mean Feature Values)")
plt.tight_layout()
plt.savefig("images/sku_cluster_profiles.png")
plt.show()


Cluster Interpretation (Example)
• Cluster 0 – Stable High Performers
– High mean sales, relatively low coefficient of variation.
– Near-flat or slightly positive trend. Good candidates for automated replenishment.

• Cluster 1 – Volatile Seasonal SKUs
– High seasonality_strength and higher cv.
– Likely tied to holidays or specific events; need more conservative inventory buffers and promo-aware forecasting.

• Cluster 2 – Declining SKUs
– Negative trend with moderate-to-low mean.
– Candidates for markdowns, assortment review, or delisting.

• Cluster 3 – Low-Volume / Noisy SKUs
– Low mean, high cv, weak trend signal.
– Hard to forecast individually; often treated with specialized intermittent-demand methods or pooled models.